In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

## **Load dataset**

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

170498071/170498071 [==============================] - 379s 2us/step


((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

## **Normalization**

In [3]:
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 225.0
X_train.dtype, X_test.dtype

(dtype('float32'), dtype('float32'))

## **Explanation**

A **Conv2D layer** is a 2D convolution layer that applies a `set of filters` to the input image. Each filter is a small matrix that slides over the image and computes a `dot product` between the filter values and the pixel values. The result is a new image (called a `feature map`) that highlights the features that the filter is designed to detect. For example, some filters may **detect edges, corners, colors, textures**, etc. The Conv2D layer can have multiple filters, each producing a different feature map. The layer also has some parameters that control how the filters are applied, such as:

- `filters`: the number of filters (or output channels) in the layer.
- `kernel_size`: the size of each filter (or kernel) in the layer. It can be a single integer or a tuple of two - integers for height and width.
- `strides`: the number of pixels that the filter moves by in each dimension. It can be a single integer or a tuple of two integers for vertical and horizontal strides.
- `padding`: how to handle the edges of the input image. It can be either “valid” or “same”. **“valid”** means no padding is added and the output size may be smaller than the input size. **“same”** means padding is added such that the output size is the same as the input size.
- `activation`: an optional function to apply to the output of the layer, such as “relu”, “sigmoid”, “tanh”, etc.

A **MaxPooling2D layer** is a 2D pooling layer that **reduces the size of the input image** by taking the maximum value over a window of pixels. This helps to reduce noise, increase speed, and extract dominant features from the image. The MaxPooling2D layer also has some parameters that control how the pooling is done, such as:

- `pool_size`: the size of the window over which to take the maximum. It can be a single integer or a tuple of two integers for height and width.
- `strides`: the number of pixels that the window moves by in each dimension. It can be a single integer or a tuple of two integers for vertical and horizontal strides. If None, it defaults to pool_size.
- `padding`: how to handle the edges of the input image. It can be either “valid” or “same”. “valid” means no padding is added and the output size may be smaller than the input size. “same” means padding is added such that the output size is the same as the input size.

**How to calculate the ouput size after applying convolution?**

output_size = (input_size - kernel_size + 2 * padding) / stride + 1 = (32 - 3 + 2 * 0) / 1 + 1 = 30

- padding = `valid` -> padding = 0
- padding = `same`  -> padding = (kernel_size - 1) / 2.

In [6]:
model_cifar10 = keras.Sequential(
    [
        keras.Input(shape=(32,32,3)), 
        layers.Conv2D(32, 3, padding='valid', activation='relu'), # 32 filters, each with kernel of size (3,3)
        layers.MaxPooling2D(pool_size=(2,2)), # Each pixel in this feature map represents the maximum value in a 2x2 region of the input feature map.
        layers.Conv2D(64, 3, activation='relu'), # 64 filters, each with kernel of size (3,3)
        layers.MaxPooling2D(), # pool_size=(2,2) by default
        layers.Conv2D(128, 3, activation='relu'), # 128 filters, each with kernel of size (3,3)
        layers.Flatten(), # reshape the input feature map into a one-dimensional vector -> prepare the input for the dense layers
        layers.Dense(64, activation='relu'), # A Dense layer with 64 units
        layers.Dense(10), # computes a linear transformation
    ]
)

model_cifar10.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)             